In [2]:
import pandas as pd
import numpy as np
def write_submission_file(_model, _featureList): # see submission.ipynb for details
    import pandas as pd
    sample = pd.read_csv('data/sample.csv', index_col=0)
    Xsample = sample[_featureList].values
    sample['Pred'] = _model.predict_proba(Xsample)[:,0]
    submission = sample[['Id', 'Pred']]
    submission.to_csv("data/submission.csv", encoding='ascii')
    
tourney_df = pd.read_csv("data/data.csv", index_col=0)
# use the recent five years of data, which is closer to what kaggle requires
BM_tourney_df=tourney_df[(tourney_df['Season']>2010) & (tourney_df['Season']<2015)]
BM_tourney_df.head()
#print BM_tourney_df.shape

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,std_stl0,std_stl1,avg_blk0,avg_blk1,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1
512,2011,134,1155,70,1412,52,N,0,26,50,...,2.64,2.44,3.13,4.88,1.81,2.38,16.80,18.38,4.20,3.80
513,2011,134,1421,81,1114,77,N,1,27,54,...,2.23,2.35,4.38,2.36,1.93,1.43,19.66,20.73,4.04,4.40
514,2011,135,1427,70,1106,61,N,0,23,54,...,2.68,2.68,4.56,2.38,2.58,1.42,22.31,19.76,4.53,5.06
515,2011,135,1433,59,1425,46,N,0,20,59,...,2.30,3.20,2.97,3.68,2.16,1.87,17.24,18.24,4.12,3.81
516,2011,136,1139,60,1330,58,N,0,22,54,...,2.35,3.20,4.36,1.52,2.77,1.21,14.73,19.16,3.71,4.49


# Get SpreadBetting data 
Add in FeatureList for the bench mark data

In [3]:
tourney_seeddf = pd.read_csv('data/TourneySeedBetting.csv')
#need two features to run SVM, let's use the seeds difference first. 
def spreadbet(game):
    
    points=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]['spreadbet'].iloc[0]
    #print points,game['team0Win']
    #The spreadbets in original table was defined as the bets of the Wteam versus Lteam. 
    #We rescale spreadbet by dividing everynumber by 10.
    if game['team0']==game['Wteam']:
        return (points)/10.
    else:
        return -(points)/10.
    

def seeddiff(game):
    
    Wseed=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]['Wseed']
    Lseed=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]['Lseed']
    #print Wseed.iloc[0],Lseed.iloc[0],game['team0Win']
    #We rescale the seeddiff by dividing 8. 
    if game['team0']==game['Wteam']:
        return (Wseed.iloc[0]-Lseed.iloc[0])/8.
    else:
        return (Lseed.iloc[0]-Wseed.iloc[0])/8.
    #return abs((Wseed.iloc[0]-Lseed.iloc[0])/8.)
    
def meanprob(game):
    
    game_seeddf=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]
    prob=game_seeddf['meanProb'].iloc[0]
    Wseed=game_seeddf['Wseed'].iloc[0]
    Lseed=game_seeddf['Lseed'].iloc[0]
    
    #print points,game['team0Win']
    #print prob,game['team0'],game['team1']
    #the meanprob in the table is defined as the small seed team win against the large seed team.
    #this does not need to be rescaled because it is already between 0 and 1 
    if game['team0']==0:
        if Wseed < Lseed:
            return 1-prob
        else:
            return prob
    else:
        if Wseed < Lseed:
            return prob
        else:
            return 1-prob
        
tourney_seeddf.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wseed,Lseed,meanProb,numProb,spreadbet
0,2011,134,1155,70,1412,52,N,0,12,12,0.50,1,0
1,2011,134,1421,81,1114,77,N,1,16,16,0.50,1,0
2,2011,135,1427,70,1106,61,N,0,16,16,0.50,1,0
3,2011,135,1433,59,1425,46,N,0,11,11,0.50,1,0
4,2011,136,1139,60,1330,58,N,0,8,9,0.51,124,1


In [4]:
fList=[spreadbet,seeddiff,meanprob]
#spreadbet(BM_tourney_df.iloc[5])
#BM_tourney_df.iloc[5:10].apply(spreadbet,axis=1).shape
#print BM_tourney_df.shape
for feature in fList:
    BM_tourney_df[feature.__name__] = BM_tourney_df.apply(feature, axis=1)
BM_tourney_df.head(20)

/Users/dtamayo/miniconda2/envs/ml2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,avg_blk1,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1,spreadbet,seeddiff,meanprob
512,2011,134,1155,70,1412,52,N,0,26,50,...,4.88,1.81,2.38,16.80,18.38,4.20,3.80,-0.00,0.000,0.50
513,2011,134,1421,81,1114,77,N,1,27,54,...,2.36,1.93,1.43,19.66,20.73,4.04,4.40,0.00,0.000,0.50
514,2011,135,1427,70,1106,61,N,0,23,54,...,2.38,2.58,1.42,22.31,19.76,4.53,5.06,-0.00,0.000,0.50
515,2011,135,1433,59,1425,46,N,0,20,59,...,3.68,2.16,1.87,17.24,18.24,4.12,3.81,-0.00,0.000,0.50
516,2011,136,1139,60,1330,58,N,0,22,54,...,1.52,2.77,1.21,14.73,19.16,3.71,4.49,-0.10,0.125,0.51
517,2011,136,1140,74,1459,66,N,0,24,61,...,2.48,2.14,1.56,17.61,19.21,4.06,4.01,-0.85,-1.375,0.84
518,2011,136,1153,78,1281,63,N,0,29,54,...,3.79,2.09,1.85,20.59,17.30,3.94,3.70,0.10,0.625,0.65
519,2011,136,1163,81,1137,52,N,0,32,66,...,3.18,2.93,2.17,16.03,18.24,3.66,4.17,-1.00,-1.375,0.84
520,2011,136,1196,79,1364,51,N,0,29,53,...,3.55,2.99,1.75,19.69,14.42,3.69,3.35,1.25,1.625,0.94
521,2011,136,1211,86,1385,71,N,0,28,52,...,2.81,2.17,1.49,18.69,18.88,4.23,4.31,0.10,0.625,0.35


# Choosing Features to Use for Model

Edit featureList below to use any of the features in the data frame above (see above for names of columns).  

In [5]:
#featureList = ['team0','team1','seeddiff','spreadbet']
featureList = ['seeddiff','spreadbet','meanprob','wins0','wins1']

y = BM_tourney_df['team0Win'].values # results
X = BM_tourney_df[featureList].values # features
print BM_tourney_df['wins0'].max(), BM_tourney_df['wins0'].min(), BM_tourney_df['wins0'].median()

#I filted out where spreadbet==0, this is what I used to identify the first four games. 
index=X[:,1]==0
y=y[~index]
X=X[~index,:]
# rescale the wins0 and wins1 features
X[:,3]=(X[:,3]-24.)/(20.)
X[:,4]=(X[:,4]-24.)/(20.)
print X.shape

32 11 24.0
(251, 5)


# Train our model!

In [6]:
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=4)

In [7]:
from sklearn import svm
from sklearn import metrics

#grid search for the best parameters, only do it once, how to do this optimized with logloss?
if (1):
    C_range = np.logspace(-2, 10, 13)
    gamma_range = np.logspace(-9, 3, 13)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.2, random_state=42)
    grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)
    grid.fit(X_train, y_train) 
    print "The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_) 
   
model = svm.SVC(probability=True,gamma=grid.best_params_['gamma'],C=grid.best_params_['C'])
#model = svm.SVC(probability=True,gamma=10,C=1)
#model = svm.SVC(probability=True)
model.fit(X_train, y_train)

The best parameters are {'C': 1.0, 'gamma': 10.0} with a score of 0.89


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=10.0, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
y_pred = model.predict_proba(X_test) # probability that team0 wins (what Kaggle calls team 1, and wants for submission)
for i in range(5):
    print("Features = {0}, y = {1}, Pred. prob. team0 wins: {2}".format(X_test[i], y_test[i], y_pred[i]))

Features = [ 1.    0.1   0.66 -0.05  0.25], y = 0, Pred. prob. team0 wins: [ 0.73057652  0.26942348]
Features = [ 0.625  0.45   0.65  -0.05  -0.05 ], y = 0, Pred. prob. team0 wins: [ 0.84876798  0.15123202]
Features = [ 0.125  0.35   0.55  -0.15  -0.05 ], y = 0, Pred. prob. team0 wins: [ 0.28006856  0.71993144]
Features = [ 0.875  0.2    0.65   0.25  -0.15 ], y = 0, Pred. prob. team0 wins: [ 0.55485127  0.44514873]
Features = [-0.875 -0.55   0.81   0.35  -0.15 ], y = 1, Pred. prob. team0 wins: [ 0.11587382  0.88412618]


# Calculating logloss 

Now the target vector is modified to be team0Win, when team0Win, y=1, which is consistant with the Koggle definition and the github definition. Please compare http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html and https://www.kaggle.com/c/march-machine-learning-mania-2016/details/evaluation to make sure we're doing the same thing.

In [9]:

print("Log loss is {0}".format(metrics.log_loss(y_test, y_pred)))

Log loss is 0.345324062582


## Look at which game voilates our expectation instead

In [118]:
#featureList = ['seeddiff','spreadbet']
#y = BM_tourney_df['team0Win'].values # results
#X = BM_tourney_df[featureList].values # features
y1 = np.zeros(len(y))
y1[(X[:,0]<0)* (y==0)]=1
y1[(X[:,0]>0)* (y==1)]=1
print X.shape,y1.shape
X_train, X_test, y_train, y_test = train_test_split(X, y1, train_size=0.8, random_state=4)
model = svm.SVC(probability=True,C=1,gamma=10)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
print("Log loss is {0}".format(metrics.log_loss(y_test, y_pred)))

(251, 5) (251,)
Log loss is 0.244687288061


# Making a submission on Kaggle

The following call generates a file `data/submission.csv` that uses the model you pass it to generate probabilities for all the matches Kaggle wants us to predict.  It will overwrite any previous file.  See `submission.ipynb` for a walk-through of the function.  Have to pass it the model you trained, and the featureList you used.

In [ ]:
write_submission_file(model, featureList)